In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt

import hoomd
import gsd.hoomd
import coxeter

from alchemy import AlchemUpdater
from harmonic import HarmonicUpdater

#### initializing snapshot

In [2]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu, seed=9520)
sim.create_state_from_gsd(filename='./DATA/lattice.gsd')

#### initializing integrator

In [3]:
alpha_init = 0.5
platonic = coxeter.families.TruncatedTetrahedronFamily()
particle = platonic.get_shape(truncation=alpha_init)
verts = particle.vertices/particle.volume**(1/3)

mc = hoomd.hpmc.integrate.ConvexPolyhedron()
mc.shape["A"] = dict(vertices=verts)

#### harmonic constraint

In [4]:
lattice = gsd.hoomd.open('./DATA/lattice.gsd', 'rb')
pos = lattice[0].particles.position
ori = lattice[0].particles.orientation
box = lattice[0].configuration.box

k_init=30
k_second=20

k_rot_init=10
k_rot_second=0

t_ramp=int(1e5)
k_trans = hoomd.variant.Power(A=k_init, 
                              B=k_second,
                              power=1,
                              t_start=sim.timestep, 
                              t_ramp=t_ramp)
k_rot = hoomd.variant.Power(A=k_rot_init, 
                            B=k_rot_second,
                            power=1,
                            t_start=sim.timestep, 
                            t_ramp=t_ramp)

harmonic = hoomd.hpmc.external.field.Harmonic(reference_positions=pos,
                                              reference_orientations=ori,
                                              k_translational=k_trans,
                                              k_rotational=k_rot,
                                              symmetries=ori)
mc.external_potential=harmonic

#### harmonic updater

In [5]:
harmonicupdater = hoomd.update.CustomUpdater(action=HarmonicUpdater(ref_pos=pos, 
                                                                    init_size=box[0]), trigger=1)

#### constant pressure updater

In [6]:
boxmc = hoomd.hpmc.update.BoxMC(trigger=hoomd.trigger.Periodic(10),
                                   betaP=15.0)

boxmc.volume['weight']=1.0
boxmc.volume['mode']='standard'
boxmc.volume['delta']=0.1

tune_boxmc = hoomd.hpmc.tune.BoxMCMoveSize.scale_solver(boxmc=boxmc,
                                                  trigger=hoomd.trigger.Periodic(100),
                                                  moves=['volume'],
                                                  target=0.2,
                                                  max_move_size={'volume':1})

#### alchemical updater

In [7]:
rng = np.random.default_rng(1234) # random generator
alchemupdater = hoomd.update.CustomUpdater(action=AlchemUpdater(stepsize=0.001, 
                                                                rng=rng,
                                                                alpha_init=alpha_init), trigger=1)


#### logger for writing gsd

In [8]:
logger = hoomd.logging.Logger()
logger.add(mc, quantities=['type_shapes'])
logger.add(alchemupdater, quantities=['alpha'])

gsd_writer = hoomd.write.GSD(filename='./DATA/trajectory.gsd',
                             trigger=hoomd.trigger.Periodic(100),
                             mode='wb',
                             filter=hoomd.filter.All())
gsd_writer.log=logger

/Users/lina492375qw1188/opt/miniconda3/envs/alchemy/lib/python3.9/site-packages/hoomd/write/gsd.py:286: FutureWarning: log property is deprecated since v3.9.0. Use logger instead.
  warnings.warn(


#### attaching operations

In [9]:
sim.operations.writers.append(gsd_writer)
sim.operations.integrator = mc
sim.operations.updaters.append(alchemupdater)
sim.operations.updaters.append(harmonicupdater)
sim.operations.updaters.append(boxmc)
sim.operations.tuners.append(tune_boxmc)

#### run simulation

In [10]:
start = time.time()

sim.run(1e5)

print('Time elapsed', time.time()-start)

/Users/lina492375qw1188/opt/miniconda3/envs/alchemy/lib/python3.9/site-packages/hoomd/data/collections.py:241: IsolationWarning: The data structure is removed from its original data source, and updates will no longer modify the previously composing object. Call obj.to_base() to remove this warning.
  warnings.warn(hoomd.error.IsolationWarning())


Time elapsed 1090.446980714798


#### check overlaps

In [11]:
mc.overlaps, alchemupdater.alchem_moves

(0, (82806, 17194))